In [1]:
from model.model import *
from processing_pipeline import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
download()
alldat = load()
print("Data Loaded. Proceeding to Preprocessing...")
session = 12

Loading data. Patience, padawan ...
Data Loaded. Proceeding to Preprocessing...


In [4]:
dfs = pd.concat([preprocess(alldat, i) for i in range(39)])

In [5]:
train, test = train_test_split(dfs)

In [6]:
train.shape

(7814, 21)

In [7]:
set(test["session"])

{1, 4, 8, 12, 19, 22, 25, 30, 35}

In [8]:
df = train

In [10]:
# -------------------------- FEATURES -------------------------- 
from scipy.special import expit
total_time = 2 # in s

conf_pred = np.empty([len(df['resp_time']),])
surprise = np.empty([len(df['resp_time']),])
expected_performance = np.empty([len(df['resp_time']),])
latency = df['resp_time']-df['gocue']
confidence = np.squeeze(1 - ((latency)/total_time))
actual_performance = np.squeeze((df['feedback_type']> 0).astype(float))
difficulty = 1 - np.abs(df['contrast_right']-df['contrast_left'])
# Set initial confidence
conf_pred[0] = confidence[0]

# Convert into trial-feature matrix
past_confidence = confidence[:-1]
current_confidence = confidence[1:]

KeyError: 'gocue'

In [13]:
from sklearn.metrics import mean_squared_error as mse 
from scipy.special import expit
total_time = 2 # in s
alpha = np.linspace(0,1,10)
unique_sessions = np.unique(df['session'])
mse_model = np.empty([len(alpha),len(unique_sessions)])


for sess_ind, session in enumerate(unique_sessions):
    curr_df = df[df['session']==session]
    conf_pred = np.empty([len(curr_df['resp_time']),])
    surprise = np.empty([len(curr_df['resp_time']),])
    expected_performance = np.empty([len(curr_df['resp_time']),])
    latency = curr_df['latency']
    confidence = np.squeeze(1 - ((latency)/total_time))
    actual_performance = np.squeeze((curr_df['feedback_type']> 0).astype(float))
    difficulty = 1 - np.abs(curr_df['contrast_right']-df['contrast_left'])
    # Set initial confidence
    confidence = confidence.reset_index()
    actual_performance = actual_performance.reset_index()

    print(confidence[0][0])
    conf_pred[0] = confidence[0][0]

    # Convert into trial-feature matrix
    past_confidence = confidence[:-1]
    current_confidence = confidence[1:]
    # -------------------------- MODEL -------------------------- 
    for a_ind,a in enumerate(alpha):
        # Compute predicted confidence
        for t in range(len(confidence)-1):
            expected_performance[t+1] = (1-difficulty[t+1])*conf_pred[t]
            surprise[t+1] = actual_performance[t+1] - expected_performance[t+1]
            conf_pred[t+1] = expit(conf_pred[t] + a*(surprise[t+1]))

        # -------------------------- MSE Analysis -------------------------- 
        mse_model[a_ind,sess_ind] = mse(confidence[0], conf_pred);


KeyError: 0

In [ ]:
# -------------------------- RESULTS -------------------------- 

# Plot the model results with ground truth
plt.figure(num=1, figsize=(12, 2), dpi=80, facecolor='w', edgecolor='k')
plt.plot(conf_pred,color='blue')
plt.plot(confidence,color='black')
plt.ylim([0,1]);
plt.xlim([0,100])

plt.figure(num=2, figsize=(12, 2), dpi=80, facecolor='w', edgecolor='k')
plt.plot(alpha,mse_model,color='blue')